# **MongoDB to Staging Area**

#### Import Required Libraries

In [135]:
from pymongo import MongoClient
from bson.json_util import dumps
from bson.json_util import loads
import pandas as pd
from tqdm.notebook import tqdm
from sqlalchemy import create_engine

#### Establish MongoDB Connection

In [113]:
client = MongoClient('localhost', 27017)

db = client['NBA-Stats']

player_cltn = db['players']
games_cltn = db['Games']
player_game_stats = db['player_game_stats']

#### Establish PostgreSQL Connection

In [137]:
conn = create_engine('postgresql+psycopg2://postgres:postgres\
@localhost/nba_stats')

# test connection
pd.read_sql('SELECT * FROM staging.stg_player_season_stats ORDER BY "PLAYER_ID" LIMIT 5', con=conn)

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,player_name,Category


#### Define Function to Retrieve MongoDB Query Results

In [115]:
def get_results(query):
    return loads(dumps(query))

#### Test Query Functionality

In [116]:
# Get All Relevant Data from "Player_Stats" Collection
query = get_results(player_cltn.find({}, {"resultSets": {"$slice": [0, 1]}}))

# Define Dictionary Keys as List
headers = query[0]['resultSets'][0]['headers']

# Define Dictionary Values as List
results = query[0]['resultSets'][0]['rowSet'][0]

results

[76001,
 '1990-91',
 '00',
 1610612757,
 'POR',
 23.0,
 43,
 0,
 290.0,
 55,
 116,
 0.474,
 0,
 0,
 0.0,
 25,
 44,
 0.568,
 27,
 62,
 89,
 12,
 4,
 12,
 22,
 39,
 135]

#### Initialize Dictionary with First Row

In [117]:
res = {}
for key in headers:
            # res[key] = results[vali]
            for value in results:
                res[key] = value
                results.remove(value)
                break
res

{'PLAYER_ID': 76001,
 'SEASON_ID': '1990-91',
 'LEAGUE_ID': '00',
 'TEAM_ID': 1610612757,
 'TEAM_ABBREVIATION': 'POR',
 'PLAYER_AGE': 23.0,
 'GP': 43,
 'GS': 0,
 'MIN': 290.0,
 'FGM': 55,
 'FGA': 116,
 'FG_PCT': 0.474,
 'FG3M': 0,
 'FG3A': 0,
 'FG3_PCT': 0.0,
 'FTM': 25,
 'FTA': 44,
 'FT_PCT': 0.568,
 'OREB': 27,
 'DREB': 62,
 'REB': 89,
 'AST': 12,
 'STL': 4,
 'BLK': 12,
 'TOV': 22,
 'PF': 39,
 'PTS': 135}

#### Construct Script to Load All Data From "Player_Stats" Collection to Pandas DataFrame

In [119]:
# define DataFrame with test data
df = pd.DataFrame(res, index=[0])

# redefine "query" variable
query = get_results(player_cltn.find({}, {"resultSets": {"$slice": [0, 1]}}))

# redefine "headers" variable
headers = query[0]['resultSets'][0]['headers']
headers.append('player_name')

# Use tqdm to view upload progress
for player in tqdm(query, ):
    for results in player['resultSets'][0]['rowSet']:
        results.append(player['player name'])
        res = {}
        for key in headers:
            for value in results:
                res[key] = value
                results.remove(value)
                break
        df = pd.concat([df, pd.DataFrame(res, index=[0])])
        
temp_player_stats = df
temp_player_stats

  0%|          | 0/4902 [00:00<?, ?it/s]

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,player_name
0,76001,1990-91,00,1610612757,POR,23.0,43,0,290.0,55,...,27,62,89,12,4,12,22,39,135,NaN
0,76001,1990-91,00,1610612757,POR,23.0,43,0,290.0,55,...,27,62,89,12,4,12,22,39,135,Alaa Abdelnaby
0,76001,1991-92,00,1610612757,POR,24.0,71,1,934.0,178,...,81,179,260,30,25,16,66,132,432,Alaa Abdelnaby
0,76001,1992-93,00,1610612749,MIL,25.0,12,0,159.0,26,...,12,25,37,10,6,4,13,24,64,Alaa Abdelnaby
0,76001,1992-93,00,1610612738,BOS,25.0,63,52,1152.0,219,...,114,186,300,17,19,22,84,165,514,Alaa Abdelnaby
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1627826,2020-21,00,1610612746,LAC,24.0,72,33,1609.0,257,...,189,330,519,90,24,62,81,187,650,Ivica Zubac
0,1627826,2021-22,00,1610612746,LAC,25.0,76,76,1852.0,310,...,217,427,644,120,36,77,114,203,785,Ivica Zubac
0,1627826,2022-23,00,1610612746,LAC,26.0,76,76,2169.0,326,...,236,520,756,77,29,98,117,219,818,Ivica Zubac
0,1627826,2023-24,00,1610612746,LAC,26.0,36,36,976.0,193,...,115,233,348,45,8,49,41,105,452,Ivica Zubac


#### Add Descriptive Column to Pandas DataFrame

In [120]:
temp_player_stats['Category'] = 'Regular Season Totals - By Season'
temp_player_stats.head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,player_name,Category
0,76001,1990-91,00,1610612757,POR,23.0,43,0,290.0,55,...,62,89,12,4,12,22,39,135,NaN,Regular Season Totals - By Season
0,76001,1990-91,00,1610612757,POR,23.0,43,0,290.0,55,...,62,89,12,4,12,22,39,135,Alaa Abdelnaby,Regular Season Totals - By Season
0,76001,1991-92,00,1610612757,POR,24.0,71,1,934.0,178,...,179,260,30,25,16,66,132,432,Alaa Abdelnaby,Regular Season Totals - By Season
0,76001,1992-93,00,1610612749,MIL,25.0,12,0,159.0,26,...,25,37,10,6,4,13,24,64,Alaa Abdelnaby,Regular Season Totals - By Season
0,76001,1992-93,00,1610612738,BOS,25.0,63,52,1152.0,219,...,186,300,17,19,22,84,165,514,Alaa Abdelnaby,Regular Season Totals - By Season


#### Perform the following Data Cleaning Tasks:
##### - Remove NULL Values (as appropriate)
##### - Change Data Types (as appropriate)
###### *I identified these steps when I was uploading the eventual CSV to the Staging Table in PostgreSQL

In [121]:
int_columns = ['GS', 'FGM', 'FGA', 'FG3M', 'FG3A', 'OREB', 'DREB', 'REB', 'STL', 'BLK', 'TOV']
float_columns = ['MIN', 'FG_PCT', 'FG3_PCT', 'FT_PCT']

for column in int_columns:
    temp_player_stats[column].fillna(value=-1, inplace=True)
    temp_player_stats[column] = temp_player_stats[column].astype(int)
    
for column in float_columns:
    temp_player_stats[column].dropna(inplace=True)
    temp_player_stats[column] = temp_player_stats[column].astype(float)


temp_player_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29727 entries, 0 to 0
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PLAYER_ID          29727 non-null  int64  
 1   SEASON_ID          29727 non-null  object 
 2   LEAGUE_ID          29727 non-null  object 
 3   TEAM_ID            29727 non-null  int64  
 4   TEAM_ABBREVIATION  29719 non-null  object 
 5   PLAYER_AGE         29727 non-null  float64
 6   GP                 29727 non-null  int64  
 7   GS                 29727 non-null  int32  
 8   MIN                28958 non-null  float64
 9   FGM                29727 non-null  int32  
 10  FGA                29727 non-null  int32  
 11  FG_PCT             29713 non-null  float64
 12  FG3M               29727 non-null  int32  
 13  FG3A               29727 non-null  int32  
 14  FG3_PCT            23472 non-null  float64
 15  FTM                29727 non-null  int64  
 16  FTA                29727 non-nu

#### Upload DataFrame to Staging Schema as "stg_player_season_stats"

In [154]:
temp_player_stats.to_sql('stg_player_season_stats', con=conn, index=False, if_exists='append', schema='staging')
print("Data uploaded successfully.")

Data uploaded successfully.


#### Gather All Team Season Data and Add Each DataFrame to a Table in the Staging Schema

In [160]:
from tqdm.notebook import tqdm

seasons = ["2003-04", "2004-05", "2006-07", "2007-08", "2008-09", "2009-10", "2010-11", "2012-13", "2013-14", "2014-15", "2015-16", "2016-17", "2017-18", "2018-19", "2019-20", "2020-21", "2021-22", "2022-23", "2023-24"]
for season in seasons:
    results = get_results(games_cltn.find({"SEASON_YEAR":season}))
    try:
        res = results[0]
        df = pd.DataFrame(res, index=[0])
    except IndexError:
        pass
    
    i = 1
    for player in tqdm(results, ):
        try:
            res = results[i]
            df = pd.concat([df, pd.DataFrame(res, index=[0])])
        except IndexError:
            pass
        i += 1
    try:
        df.drop(columns=['_id'], inplace=True)
    except:
        df.to_sql(f"stg_{season}_games", index=False, schema='staging', if_exists='replace', con=conn)
    df.to_sql(f"stg_{season}_games", index=False, schema='staging', if_exists='replace', con=conn)

  0%|          | 0/2378 [00:00<?, ?it/s]

  0%|          | 0/2460 [00:00<?, ?it/s]

  0%|          | 0/2460 [00:00<?, ?it/s]

  0%|          | 0/2460 [00:00<?, ?it/s]

  0%|          | 0/2460 [00:00<?, ?it/s]

...

  0%|          | 0/2118 [00:00<?, ?it/s]

  0%|          | 0/2160 [00:00<?, ?it/s]

  0%|          | 0/2460 [00:00<?, ?it/s]

  0%|          | 0/2460 [00:00<?, ?it/s]

  0%|          | 0/1196 [00:00<?, ?it/s]

#### Gather All Player Game Stats from API

In [178]:
query = get_results(db.player_game_stats.find({}))

#### Collate All Query Results Into DataFrames and Upload Each DataFrame to the Staging Schema as the "stg_player_game_stats" Table

In [202]:

df_concatenated1 = pd.concat([pd.DataFrame(row, index=[0]).drop(columns=['_id']) for row in tqdm(query[:100000],)], ignore_index=True)
df_concatenated1.to_sql('stg_player_game_stats', con=conn, index=False, schema='staging', if_exists='replace')
print('Batch 1 uploaded')
df_concatenated2 = pd.concat([pd.DataFrame(row, index=[0]).drop(columns=['_id']) for row in tqdm(query[100000:200000],)], ignore_index=True)
df_concatenated2.to_sql('stg_player_game_stats', con=conn, index=False, schema='staging', if_exists='append')
print('Batch 2 uploaded')
df_concatenated3 = pd.concat([pd.DataFrame(row, index=[0]).drop(columns=['_id']) for row in tqdm(query[200000:300000],)], ignore_index=True)
df_concatenated3.to_sql('stg_player_game_stats', con=conn, index=False, schema='staging', if_exists='append')
print('Batch 3 uploaded')
df_concatenated4 = pd.concat([pd.DataFrame(row, index=[0]).drop(columns=['_id']) for row in tqdm(query[300000:400000],)], ignore_index=True)
df_concatenated4.to_sql('stg_player_game_stats', con=conn, index=False, schema='staging', if_exists='append')
print('Batch 4 uploaded')
df_concatenated5 = pd.concat([pd.DataFrame(row, index=[0]).drop(columns=['_id']) for row in tqdm(query[400000:],)], ignore_index=True)
df_concatenated5.to_sql('stg_player_game_stats', con=conn, index=False, schema='staging', if_exists='append')
print('Batch 5 uploaded')

  0%|          | 0/100000 [00:00<?, ?it/s]

Batch 1 uploaded


  0%|          | 0/100000 [00:00<?, ?it/s]

Batch 2 uploaded


  0%|          | 0/100000 [00:00<?, ?it/s]

Batch 3 uploaded


  0%|          | 0/100000 [00:00<?, ?it/s]

Batch 4 uploaded


  0%|          | 0/86529 [00:00<?, ?it/s]

Batch 5 uploaded
